In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import logging
from sklearn.ensemble import RandomForestClassifier

In [2]:
dataset = pd.read_csv('q1.csv')
dataset= dataset[dataset['ans'].notna()]#ignore NAN values
print(dataset.head())
# print(len(answer))

  img_id                                                ans  marks
0  IMG01  ଇଂଗରେଜକଂ ସାମରାଜ୍ୟ କୁ ସୁଦୃଢ କରିବା ପାଇଁ ୱେଲସେଲି ...    5.0
1  IMG02  ଇଂଗରେଜକଂ ପ୍ରଭାବିତ ରାଜ୍ୟ ବିସ୍ତାର ପାଇଁ ରାଜ୍ୟାଭିଳ...    5.0
3  IMG04  ସାମନ୍ତ ସନ୍ଧି ପ୍ରଥା ୱେଲସେଲି ପ୍ରଣୟନ କରିଥିଲେ|ଏହାର...    4.0
4  IMG05  ସାମନ୍ତ ସନ୍ଧି ଲର୍ଡ଼ ମାକଏଲେକଂ ଦ୍ୱାରା ପ୍ରଣୀତ ହୋଇଥ...    1.0
5  IMG06  ଭାରତରେ ଇଂଗ୍ରେଜ଼ ମାନଂକ ସାମ୍ରାଜ୍ୟ ବିସ୍ତାର ପାଇଁ ୱ...    5.0


In [3]:
#classify training and test set
from sklearn.model_selection import train_test_split
train,test = train_test_split(dataset,test_size=0.15,random_state=42)

In [4]:
print(train.head())

   img_id                                                ans  marks
76  IMG77  ସମାନତା ସନ୍ଧି ପ୍ରଥା ଜେନେରାଲ ଡେଲହାଉସି ପ୍ରଣୟନ କରି...    4.5
39  IMG40  ଭାରତରେ ଇଂଗ୍ରେଜ଼ ମାନଙ୍କ ପ୍ରଭାବ ବିସ୍ତାର କରିବା ପା...    4.0
0   IMG01  ଇଂଗରେଜକଂ ସାମରାଜ୍ୟ କୁ ସୁଦୃଢ କରିବା ପାଇଁ ୱେଲସେଲି ...    5.0
79  IMG80  ସାମନ୍ତ ସନ୍ଧି ୱେଲସେଲି ପ୍ରଣୟନ କରିଥଲେ ଇଂଗ୍ରେଜ଼ ମା...    4.5
46  IMG47  ସାମନ୍ତ ସନ୍ଧିକୁ ଇଂଗ୍ରେଜ଼ ଲର୍ଡ଼ ଡେଲହାଉସି ପ୍ରଚଳନ ...    0.5


In [5]:
print(test)

   img_id                                                ans  marks
48  IMG49  ଭାରତରେ ସାମ୍ରାଜ୍ୟ ବିସ୍ତାର କରିବ ତଥା ବାଣିଜ୍ୟ ପ୍ରସ...    2.0
21  IMG22  ଇଂଗ୍ରେଜ଼ ମାନଂକ ବିନା ଅନୁମତିରେ କୌଣସି ଦେଶ ସହ ବାଣି...    3.0
5   IMG06  ଭାରତରେ ଇଂଗ୍ରେଜ଼ ମାନଂକ ସାମ୍ରାଜ୍ୟ ବିସ୍ତାର ପାଇଁ ୱ...    5.0
10  IMG11  ୱେଲସେଲି ସାମନ୍ତ ସନ୍ଧି ପ୍ରଚଳନ କରିଥିଲେ|ତାଙ୍କର ଅନ୍...    2.0
58  IMG59  ଭାରତରେ ଇଂଗ୍ରେଜ଼ ମାନଙ୍କ ସାମ୍ରାଜ୍ୟ ବୃଦ୍ଧି ପାଇଁ ୱ...    5.0
52  IMG53  ଭାରତରେ ଇଂଗ୍ରେଜ଼ ମାନଙ୍କ ପ୍ରଭୁତ୍ୱ ବିସ୍ତାର କାରିବା...    4.5
78  IMG79  ଇଂଗ୍ରେଜ଼ ମାନେ ସାମ୍ରାଜ୍ୟ ବୃଦ୍ଧି କରିବା ପାଇଁ ୱେଲସ...    4.0
6   IMG07  ୱେଲସେଲି ଇଂଗ୍ରେଜ଼ ଶାସନର ପ୍ରସାର ପାଇଁଏକ ପ୍ରଥା ପ୍ର...    5.0
81  IMG82  ଇଂଗ୍ରେଜ଼ ମାନେ ଭାରତରେ ବାଣିଜ୍ୟ କରିବାକୁ ଆସିଥିଲେ|ଭ...    2.0
16  IMG17  ଭାରତରେ ରାଜ୍ୟ ବିସ୍ତାର କରିବା ପାଇଁ ଲର୍ଡ଼ ୱେଲସେଲି ...    5.0
34  IMG35  ସାମନ୍ତ ସନ୍ଧି ପ୍ରଥା ପ୍ରଚଳନ କରିଥିଲେ କାରଣ ସେ ଭାରତ...    0.0


In [6]:
from inltk.inltk import tokenize,get_embedding_vectors #importing tokenizer using inltk
def sentence_tokenization(answer):
    token = []
    for ans in answer:
        token.append(tokenize(str(ans),'or'))
    return token

In [7]:
answer = []
for ans in train['ans']:
    answer.append(ans)
token=[]
print("Tokenization of training set")
token = sentence_tokenization(answer)
# print(token)

Tokenization of training set


In [8]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
        level=logging.INFO)

In [9]:
    num_features = 400    # Word vector dimensionality
    min_word_count = 5   # Minimum word count
    num_workers = 4       # Number of threads to run in parallel
    window = 5         # Context window size
    downsampling = 1e-3   # Downsample setting for frequent words

In [10]:
    print("Training Word2Vec model...")
    model = Word2Vec(token, workers=num_workers, \
                size=num_features, min_count = min_word_count, \
                window = window, sample = downsampling, seed=1)

    # If you don't plan to train the model Any further, calling
    # init_sims will make the model much more memory-efficient.
    model.init_sims(replace=True)
    print("MODEL",model)
    # It can be helpful to create a meaningful model name and
    # save the model for later use. You can load it later using Word2Vec.load()
    model_name = "word2vec_model"
    model.save(model_name)

2020-05-03 01:46:04,942 : INFO : collecting all words and their counts
2020-05-03 01:46:04,944 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-03 01:46:04,956 : INFO : collected 662 word types from a corpus of 6956 raw words and 56 sentences
2020-05-03 01:46:04,958 : INFO : Loading a fresh vocabulary
2020-05-03 01:46:04,965 : INFO : effective_min_count=5 retains 199 unique words (30% of original 662, drops 463)
2020-05-03 01:46:04,967 : INFO : effective_min_count=5 leaves 6186 word corpus (88% of original 6956, drops 770)
2020-05-03 01:46:04,974 : INFO : deleting the raw counts dictionary of 662 items
2020-05-03 01:46:04,976 : INFO : sample=0.001 downsamples 93 most-common words
2020-05-03 01:46:04,977 : INFO : downsampling leaves estimated 3157 word corpus (51.0% of prior 6186)
2020-05-03 01:46:04,979 : INFO : estimated required memory for 199 words and 400 dimensions: 736300 bytes
2020-05-03 01:46:04,980 : INFO : resetting layer weights
2020-05-03 0

Training Word2Vec model...


2020-05-03 01:46:05,140 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-03 01:46:05,141 : INFO : EPOCH - 4 : training on 6956 raw words (3110 effective words) took 0.0s, 241020 effective words/s
2020-05-03 01:46:05,146 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-03 01:46:05,147 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-03 01:46:05,147 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-03 01:46:05,155 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-03 01:46:05,156 : INFO : EPOCH - 5 : training on 6956 raw words (3096 effective words) took 0.0s, 276796 effective words/s
2020-05-03 01:46:05,156 : INFO : training on a 34780 raw words (15606 effective words) took 0.1s, 168289 effective words/s
2020-05-03 01:46:05,157 : WARNING : under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2020-05-03 01:46:05,158 : INFO :

MODEL Word2Vec(vocab=199, size=400, alpha=0.025)


In [11]:
def featureVec(words, model, num_features):
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    #
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the ans and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    #
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
#     print(featureVec)
    return featureVec

In [12]:
def avgFeatureVecs(reviews, model, num_features):
    # Initialize a counter
    counter = 0.
    #
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    #
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 10th review
       if counter%10. == 0.:
           print("Review %d of %d" % (counter, len(reviews)))
       #
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[int(counter)] = featureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    print(reviewFeatureVecs)
    return reviewFeatureVecs

In [13]:
trainVec = avgFeatureVecs( token, model, num_features )

Review 0 of 56
Review 10 of 56
Review 20 of 56
Review 30 of 56
Review 40 of 56
Review 50 of 56
[[-0.025879 -0.045129  0.073868 -0.022345 ...  0.041083  0.074698  0.073807 -0.065894]
 [-0.02579  -0.045233  0.073989 -0.022277 ...  0.041099  0.074681  0.073776 -0.065901]
 [-0.025756 -0.045234  0.074098 -0.022338 ...  0.041064  0.074571  0.073722 -0.06604 ]
 [-0.025845 -0.045289  0.073876 -0.022322 ...  0.041147  0.074572  0.073779 -0.065855]
 ...
 [-0.025788 -0.045248  0.073966 -0.022139 ...  0.041106  0.074482  0.073596 -0.066005]
 [-0.025817 -0.045189  0.074014 -0.022209 ...  0.041025  0.074646  0.073752 -0.065994]
 [-0.025816 -0.045179  0.073922 -0.022229 ...  0.040996  0.074652  0.073676 -0.066061]
 [-0.02583  -0.045259  0.073873 -0.022198 ...  0.041144  0.074584  0.073767 -0.065915]]


/home/iamsid2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [14]:
print(trainVec)

[[-0.025879 -0.045129  0.073868 -0.022345 ...  0.041083  0.074698  0.073807 -0.065894]
 [-0.02579  -0.045233  0.073989 -0.022277 ...  0.041099  0.074681  0.073776 -0.065901]
 [-0.025756 -0.045234  0.074098 -0.022338 ...  0.041064  0.074571  0.073722 -0.06604 ]
 [-0.025845 -0.045289  0.073876 -0.022322 ...  0.041147  0.074572  0.073779 -0.065855]
 ...
 [-0.025788 -0.045248  0.073966 -0.022139 ...  0.041106  0.074482  0.073596 -0.066005]
 [-0.025817 -0.045189  0.074014 -0.022209 ...  0.041025  0.074646  0.073752 -0.065994]
 [-0.025816 -0.045179  0.073922 -0.022229 ...  0.040996  0.074652  0.073676 -0.066061]
 [-0.02583  -0.045259  0.073873 -0.022198 ...  0.041144  0.074584  0.073767 -0.065915]]


In [15]:
len(trainVec)

56

In [16]:
def inltkWordVec(token):
    wordVec = []
    for t in token:
        wordVec.append(get_embedding_vectors(str(t),'or'))
    return wordVec

In [17]:
itrainVec = inltkWordVec(answer)
print(itrainVec)

/home/iamsid2/anaconda3/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/iamsid2/anaconda3/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'fastai.text.models.awd_lstm.AWD_LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/iamsid2/anaconda3/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code b

[[array([-0.167081,  0.2672  ,  0.354371,  0.172869, ...,  0.430621,  0.080897, -0.530237,  0.326994], dtype=float32), array([-0.341303,  0.453012,  0.210755, -0.492647, ...,  0.179599, -0.435906,  0.331539,  0.225386], dtype=float32), array([ 0.15834 , -0.323814, -0.108051,  0.591264, ..., -0.209223,  0.058719, -0.020219,  0.003265], dtype=float32), array([-0.726117,  0.432183,  0.220137,  0.302996, ..., -0.048881,  0.3892  ,  0.014426, -0.070983], dtype=float32), array([-0.770581,  0.187053, -0.221099,  0.071302, ..., -0.496939, -0.069128,  0.108977,  0.347822], dtype=float32), array([-0.750668, -0.028647,  0.328319,  0.055065, ...,  0.045931,  0.535303, -0.111293,  0.133518], dtype=float32), array([-0.236776,  0.523308,  0.26473 , -0.245358, ..., -0.105682, -0.362353,  0.000969,  0.423449], dtype=float32), array([-0.540504,  0.261988, -0.004735,  0.09172 , ..., -0.37285 , -0.371681, -0.073301,  0.298451], dtype=float32), array([-0.435258,  0.97108 , -0.205416, -0.009345, ...,  0.170

In [18]:
# for i in itrainVec:
#     print(len(i))

In [19]:
answer_test = []
for ans in test['ans']:
    answer_test.append(str(ans))
token_test=[]
print("Tokenization of test set")
token_test = sentence_tokenization(answer_test)
# print(token_test)

Tokenization of test set


In [20]:
testVec = avgFeatureVecs(token_test, model, num_features)

Review 0 of 11
Review 10 of 11
[[-0.025886 -0.045404  0.074105 -0.02226  ...  0.04112   0.074503  0.073789 -0.065888]
 [-0.025872 -0.045117  0.073822 -0.022336 ...  0.041157  0.074735  0.073794 -0.066067]
 [-0.025768 -0.045222  0.073961 -0.022208 ...  0.04113   0.074635  0.073778 -0.065949]
 [-0.02576  -0.04544   0.074198 -0.022231 ...  0.040959  0.074446  0.073661 -0.065953]
 ...
 [-0.02581  -0.045193  0.073929 -0.022238 ...  0.041078  0.074621  0.073786 -0.065964]
 [-0.025832 -0.045187  0.073991 -0.022294 ...  0.041026  0.074506  0.073692 -0.066033]
 [-0.025745 -0.045229  0.073975 -0.022272 ...  0.041089  0.074614  0.073719 -0.066092]
 [-0.025728 -0.045391  0.074018 -0.022114 ...  0.041037  0.074678  0.073647 -0.066073]]


/home/iamsid2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [21]:
testVec[0].shape

(400,)

In [22]:
itestVec = inltkWordVec(answer_test)
# print(itestVec)

In [23]:
print("Length Vec Test",len(itestVec))
print("Gensim Length")
for t in testVec:
      print(len(t))
print("INLTK Length")
for l in itestVec:
    print(len(l))

Length Vec Test 11
Gensim Length
400
400
400
400
400
400
400
400
400
400
400
INLTK Length
56
115
137
64
181
124
116
173
171
141
45


In [24]:
def pad(l, content, width):
    l.extend([content] * (width - len(l)))
    return l

# pad(itrainVec,0,300)

In [25]:
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit(trainVec, train["marks"].astype(int))

# Test & extract results
# result = forest.predict(np.array(xtestVec,dtype="float32"))

Fitting a random forest to labeled training data...


In [26]:
# Test & extract results
result = forest.predict(testVec)
# print(result)

[4 5 5 4 5 3 4 5 5 5 5]


In [27]:
#Calculate accuracy

test_mark = []
for i in test['marks'].astype(int):
    test_mark.append(i)
print("Actual Mark")
test_mark = np.array(test_mark)
print(test_mark)

from sklearn.metrics import accuracy_score
accuracy_score(test_mark, result)

0.45454545454545453